In [1]:
import os
print(os.getcwd())

/mnt/c/Users/Komar/Lab/retroparty


In [2]:
os.chdir('/mnt/c/Users/Komar/Lab')
os.getcwd()

'/mnt/c/Users/Komar/Lab'

In [3]:
import pandas as pd

In [4]:
df1 = pd.read_table('SRR7624520+test___R1_blat_out.csv', sep=',')
df1 = df1.drop(['Unnamed: 0'], axis=1)
print(df1.shape)
df1.head()

(1626, 11)


,QUERY,SCORE,QSTART,QEND,QSIZE,IDENTITY,CHROM,STRAND,START,END,SPAN
0,SRR7624520.1085275.1_R1,33,1,33,33,100.0%,chr1,+,74835989,74836021,33
1,SRR7624520.126918.1_R1,52,5,56,56,100.0%,chr1,+,247065180,247065231,52
2,SRR7624520.126918.1_R1,35,5,56,56,87.9%,chr19,+,11739528,11739578,51
3,SRR7624520.126918.1_R1,22,5,28,56,95.9%,chr19,-,11638874,11638897,24
4,SRR7624520.126918.1_R1,20,16,35,56,100.0%,chr3,+,25191224,25191243,20


In [5]:
df2 = pd.read_table('SRR7624520+test___R2_blat_out.csv', sep=',')
df2 = df2.drop(['Unnamed: 0'], axis=1)
print(df2.shape)
df2.head()

(1741, 11)


,QUERY,SCORE,QSTART,QEND,QSIZE,IDENTITY,CHROM,STRAND,START,END,SPAN
0,SRR7624520.1085275.1_R2,33,1,33,33,100.0%,chr1,+,74835989,74836021,33
1,SRR7624520.126918.1_R2,52,5,56,56,100.0%,chr1,+,247065180,247065231,52
2,SRR7624520.126918.1_R2,35,5,56,56,87.9%,chr19,+,11739528,11739578,51
3,SRR7624520.126918.1_R2,22,5,28,56,95.9%,chr19,-,11638874,11638897,24
4,SRR7624520.126918.1_R2,20,16,35,56,100.0%,chr3,+,25191224,25191243,20


In [14]:
read_names = df1['QUERY'].unique()
for name in read_names:
    df1_1 = df1[df1['QUERY'].str.contains(name)].copy(deep=True)
    df2_1 = df2[df2['QUERY'].str.contains(name[:-3])].copy(deep=True)
    merget_df = pd.DataFrame(columns=['QUERY', 
                                      'SCORE_1', 'QSIZE_1', 'QSTART_1', 'QEND_1', 'IDENTITY_1', 'CHROM_1', 'STRAND_1', 'START_1', 'END_1', 'SPAN',
                                      'SCORE_2', 'QSIZE_2', 'QSTART_2', 'QEND_2', 'IDENTITY_2', 'CHROM_2', 'STRAND_2', 'START_2', 'END_2', 'SPAN']
                            )
    for i, row_1 in df1_1.iterrows():
        tmp_df = df2_1[df2_1['CHROM'] == row_1['CHROM']]
        for i, row_tmp in tmp_df.iterrows():
            if abs(row_tmp['START'] - row_1['START']) <= 1000 and abs(row_tmp['END'] - row_1['END']) <= 1000:
                merget_df.loc[len(merget_df.index)] = [row_1['QUERY'][:-3], 
                                                       row_1['SCORE'], row_1['QSIZE'], row_1['QSTART'], row_1['QEND'], row_1['IDENTITY'],
                                                       row_1['CHROM'], row_1['STRAND'], row_1['START'], row_1['END'], row_1['SPAN'],
                                                       row_tmp['SCORE'], row_tmp['QSIZE'], row_tmp['QSTART'], row_tmp['QEND'], row_tmp['IDENTITY'], 
                                                       row_tmp['CHROM'], row_tmp['STRAND'], row_tmp['START'], row_tmp['END'], row_tmp['SPAN'] 
                                                      ]
    merget_df = merget_df.drop(merget_df[merget_df['CHROM_1'].str.contains('alt')].index, axis=0)
    merget_df = merget_df.drop(merget_df[merget_df['CHROM_2'].str.contains('alt')].index, axis=0)
    if merget_df.shape[0] > 1:
        merget_df = merget_df[merget_df['SCORE_1'] > 0.8 * merget_df['QSIZE_1']]
    if merget_df.shape[0] > 1:
        merget_df = merget_df[merget_df['SCORE_2'] > 0.8 * merget_df['QSIZE_2']]
    if merget_df.shape[0] > 1:
        merget_df = merget_df[merget_df['QSTART_1'] < 3]

    print(df1_1.shape[0], df2_1.shape[0], merget_df.shape[0])
merget_df.head()

1 1 1
4 4 1
183 182 1
198 198 143
173 189 1
138 138 51
2 2 1
203 199 2
209 209 203
12 18 1
2 2 1
10 10 2
15 15 1
2 1 1
38 34 1
2 3 1
6 7 1
202 202 0
6 6 1
5 2 1
3 1 1
90 0 0
7 1 1
115 115 79


,QUERY,SCORE_1,QSIZE_1,QSTART_1,QEND_1,IDENTITY_1,CHROM_1,STRAND_1,START_1,END_1,...,SCORE_2,QSIZE_2,QSTART_2,QEND_2,IDENTITY_2,CHROM_2,STRAND_2,START_2,END_2,SPAN
0,SRR7624520.987629.1,38,38,1,38,100.0%,chr1,-,143214752,143214789,...,38,38,1,38,100.0%,chr1,-,143214752,143214789,38
1,SRR7624520.987629.1,38,38,1,38,100.0%,chr1,+,125179400,125179437,...,38,38,1,38,100.0%,chr1,+,125179400,125179437,38
2,SRR7624520.987629.1,37,38,1,38,100.0%,chr1,-,143225556,143225594,...,37,38,1,38,100.0%,chr1,-,143225556,143225594,39
3,SRR7624520.987629.1,36,38,1,38,97.4%,chr1,-,143262430,143262467,...,36,38,1,38,97.4%,chr1,-,143262430,143262467,38
4,SRR7624520.987629.1,36,38,1,38,97.4%,chr1,-,143240936,143240973,...,36,38,1,38,97.4%,chr1,-,143240936,143240973,38


In [66]:
tmp_2darray = []
for i, row_1 in df1_1.iterrows():
    
    tmp_df = df2_1[df2_1['CHROM'] == row_1['CHROM']]
    for i, row_tmp in tmp_df.iterrows():
        
        if abs(row_tmp['START.1'] - row_1['START.1']) <= 1200 and abs(row_tmp['END.1'] - row_1['END.1']) <= 1200:
            tmp_2darray.append([row_1['QUERY'], row_1['SCORE'], row_1['START'], row_1['END'], row_1['IDENTITY'],
                                row_1['CHROM'], row_1['STRAND'], row_1['START.1'], row_1['END.1'],
                                row_tmp['SCORE'], row_tmp['START'], row_tmp['END'], row_tmp['IDENTITY'], 
                                row_tmp['CHROM'], row_tmp['START.1'], row_tmp['END.1']])
merget_df = pd.DataFrame(tmp_2darray, columns=['QUERY', 'SCORE_1', 'START_1', 'END_1', 'IDENTITY_1', 'CHROM_1', 'STRAND_1', 'START.1_1', 'END.1_1',
                                   'SCORE_2', 'START_2', 'END_2', 'IDENTITY_2', 'CHROM_2', 'START.1_2', 'END.1_2'])
merget_df = merget_df[merget_df['SCORE_1'] > 40]
merget_df = merget_df.drop(merget_df[merget_df['CHROM_1'].str.contains('alt')].index, axis=0)
#merget_df = merget_df[merget_df['SCORE_2'] > 60]
merget_df = merget_df[merget_df['START_1'] < 3]
#merget_df.sort_values('END_1' )
print(merget_df.shape[0])
merget_df

1


,QUERY,SCORE_1,START_1,END_1,IDENTITY_1,CHROM_1,STRAND_1,START.1_1,END.1_1,SCORE_2,START_2,END_2,IDENTITY_2,CHROM_2,START.1_2,END.1_2
0,5_R1,59,1,61,95.0%,chr3,-,81813044,81813103,69,2,72,95.8%,chr3,81813044,81813113


In [15]:
def format_coords(df_row):
    low_coord = min(df_row['START_1'], df_row['START_2'], df_row['END_1'], df_row['END_2'])
    high_coord = max(df_row['START_1'], df_row['START_2'], df_row['END_1'], df_row['END_2'])
    return f"{row['CHROM_1']}:{low_coord}-{high_coord}"

for i, row in merget_df.iterrows():
    merget_df['FLANK_COORDS'] = format_coords(row)
merget_df

,QUERY,SCORE_1,QSIZE_1,QSTART_1,QEND_1,IDENTITY_1,CHROM_1,STRAND_1,START_1,END_1,...,QSIZE_2,QSTART_2,QEND_2,IDENTITY_2,CHROM_2,STRAND_2,START_2,END_2,SPAN,FLANK_COORDS
0,SRR7624520.987629.1,38,38,1,38,100.0%,chr1,-,143214752,143214789,...,38,1,38,100.0%,chr1,-,143214752,143214789,38,chr2:92081354-92081392
1,SRR7624520.987629.1,38,38,1,38,100.0%,chr1,+,125179400,125179437,...,38,1,38,100.0%,chr1,+,125179400,125179437,38,chr2:92081354-92081392
2,SRR7624520.987629.1,37,38,1,38,100.0%,chr1,-,143225556,143225594,...,38,1,38,100.0%,chr1,-,143225556,143225594,39,chr2:92081354-92081392
3,SRR7624520.987629.1,36,38,1,38,97.4%,chr1,-,143262430,143262467,...,38,1,38,97.4%,chr1,-,143262430,143262467,38,chr2:92081354-92081392
4,SRR7624520.987629.1,36,38,1,38,97.4%,chr1,-,143240936,143240973,...,38,1,38,97.4%,chr1,-,143240936,143240973,38,chr2:92081354-92081392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,SRR7624520.987629.1,31,38,1,38,92.2%,chr10,-,42077650,42077688,...,38,1,38,92.2%,chr10,-,42077650,42077688,39,chr2:92081354-92081392
86,SRR7624520.987629.1,31,38,1,38,92.2%,chr10,-,42073540,42073578,...,38,1,38,92.2%,chr10,-,42073540,42073578,39,chr2:92081354-92081392
87,SRR7624520.987629.1,31,38,1,38,97.0%,chrUn_KI270747v1,+,47337,47375,...,38,1,38,97.0%,chrUn_KI270747v1,+,47337,47375,39,chr2:92081354-92081392
89,SRR7624520.987629.1,31,38,1,38,97.0%,chr9,+,43318779,43318817,...,38,1,38,97.0%,chr9,+,43318779,43318817,39,chr2:92081354-92081392


In [89]:
merget_df[merget_df['CHROM_1']=='chr5']

,QUERY,SCORE_1,START_1,END_1,IDENTITY_1,CHROM_1,START.1_1,END.1_1,SCORE_2,START_2,END_2,IDENTITY_2,CHROM_2,START.1_2,END.1_2


In [16]:
def hamming_dist(x1, x2):
    j = 0
    for i in range(len(x1)):
        if x1[i] != x2[i]:
            j = j + 1
    return j

In [21]:
def find_overlap(str1, str2):
    if len(str1) < len(str2):
        str_tmp = str1
        str1 = str2
        str2 = str_tmp
    if len(str1) >= len(str2):
        delta = len(str1) - len(str2)
        for i in range(len(str1)):
            if delta - i >= 0:
                part1 = str1[i:len(str2) + i]
                part2 = str2
            else:
                part1 = str1[i:]
                part2 = str2[0:len(str2) + delta - i]
            # print(part1, part2)
            if hamming_dist(part1, part2) == 0:
                #print(part1, len(part1))
                return part1
        return ''

In [18]:
R1_orig_list = ['GACTCTTCTTACTTTTAAGGACACATATAATTGAAACTATTTTGGAAAACAAGAGAGCAGGGATAG',
 'TACTATAAACACTTCTATGCCAATAAACTAGAAAATCCAGAAGAAATGGATAAATTCCTGGACACATACACCCTCCCAAGAC',
 'TCTAATCCATTTTGGAAGGTTAGTCTTACATGGTTGTTGCATTCCACAACTATCGGTAGATTATCTTTAAAAAGGGAAGCAA',
 'TGGTTTTTTTTTTTTTTTTGACAGTCTTGCTGTTCCCAGGCTGGAGTGCAGTGGTGCGACCTCAG',
 'ATTTTTTTATTTTTTTTAGCCTTGCATTTGCCCGTGAGAGGAGCAGCCACACTGGAACGAG',
 'ATGAAATTTTAAAATTTTTTTCTCTGCCTAGTTGCAGACGAGGAAGTTAGTGATGAGGCACAAGAAAGATTTCAGGCAAAAC',
 'AGTTTTTTTTTTTTTTTTTGAGAGGGAGTCTCGCTCTGTCACCCAGTCTGGAGTGCAGTGGCACAATCTCAG',
 'TTTTCTTCTTTTTTAAGTCTGAATAATATTCTCTTGTTTGTATTCACCACATTTTATGTATCCATTCATCCATCAATAGACA',
 'AGAGTTAAACTCAGGCCCACTCAACTTTAACCCCCATATTCTTTCCTCTACACTACCCTTTCCACTCCAGCATGCTACAATT']

R2_rev_comp = ['CGGCCGACTCTTCTTACTTTTAAGGACACATATAATTGAAACTATTTTGGAAAACAAGAGAGCAGGGATAG',
 'AGCCTACCAACCAAAAAAAGTCCAGGACCAGACAGATTCACAGATGAATTTTACCAGAGGTACAAAGAGGAG',
 'GTTGCATTCCACAACTATCGGTAGATTATCTTTAAAAAGGGAAGCAATTTCACATTTGTCCAGCAGAGGGAG',
 'CCCGGTTTTTTTTTTTTTTTTTTTTTGACAGTCTTGCTGTTCCCAGGCTGGAGTGCAGTGGTGCGACCTCAG',
 'CGCGCCCGGCCATTTTTTTATTTTTTTTAGCCTTGCATTTGCCCGTGAGAGGAGCAGCCACACTGGAACGAG',
 'GATTTCAGGCAAAACTGCCCTCTTGAAGATGGAGGGGGCCATCTGGCAAAGAATGCAGGCAGCCTCTGCGAG',
 'TTTTTTTTTTTTTTTTTTTGAGAGGGAGTCTCGCTCTGTCACCCAGGCTGGAGTGCAGTGGCACACTCTCAG',
 'TCTTGTTTGTATTCACCACATTTTATGTATCCATTCATCCATCAATAGACACTTGATTGATTCCACTTTTAG',
 'ATTCTGAGCATTTCCTGAGTATCCCTGGGGGGTGCAGAACTGTCCTAAGTACTAAGTGAAGAACAGAAGGAG']

In [22]:
for i in range(len(R1_orig_list)):

    print(f'-----{i+1}-----')
    print(len(R1_orig_list[i]))
    print(len(R2_rev_comp[i]))
    overlap = find_overlap(R1_orig_list[i], R2_rev_comp[i])
    print(overlap, len(overlap))

-----1-----
66
71
GACTCTTCTTACTTTTAAGGACACATATAATTGAAACTATTTTGGAAAACAAGAGAGCAGGGATAG 66
-----2-----
82
72
no overlap 10
-----3-----
82
72
GTTGCATTCCACAACTATCGGTAGATTATCTTTAAAAAGGGAAGCAA 47
-----4-----
65
72
no overlap 10
-----5-----
61
72
ATTTTTTTATTTTTTTTAGCCTTGCATTTGCCCGTGAGAGGAGCAGCCACACTGGAACGAG 61
-----6-----
82
72
GATTTCAGGCAAAAC 15
-----7-----
72
72
no overlap 10
-----8-----
82
72
TCTTGTTTGTATTCACCACATTTTATGTATCCATTCATCCATCAATAGACA 51
-----9-----
82
72
ATT 3


In [31]:
from random import randint
def gen_rand_num_list(min, max, length):
    if length > max:
        return None
    rand_list = []
    for i in range(int(length)): 
        rand_num = randint(min, max)
        while rand_list.count(rand_num) != 0:
            rand_num = randint(min, max)
        rand_list.append(rand_num) 
    return rand_list

In [33]:
rand_list = gen_rand_num_list(0, 11, 11-5)
rand_list

[9, 5, 10, 3, 6, 2]